<a href="https://colab.research.google.com/github/malinphy/q_17/blob/main/sbert_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers -q

In [2]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time


# # Model for computing sentence embeddings. We use one trained for similar questions detection
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # We donwload the Quora Duplicate Questions Dataset (https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs)
# # and find similar question in it
# url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
# dataset_path = "quora_duplicate_questions.tsv"
# max_corpus_size = 50000  # We limit our corpus to only the first 50k questions


# # Check if the dataset exists. If not, download and extract
# # Download dataset if needed
# if not os.path.exists(dataset_path):
#     print("Download dataset")
#     util.http_get(url, dataset_path)

# # Get all unique sentences from the file
# corpus_sentences = set()
# with open(dataset_path, encoding='utf8') as fIn:
#     reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
#     for row in reader:
#         corpus_sentences.add(row['question1'])
#         corpus_sentences.add(row['question2'])
#         if len(corpus_sentences) >= max_corpus_size:
#             break

In [3]:
# corpus_sentences = list(corpus_sentences)
# print("Encode the corpus. This might take a while")
# corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)


In [4]:
# print("Start clustering")
# start_time = time.time()

# #Two parameters to tune:
# #min_cluster_size: Only consider cluster that have at least 25 elements
# #threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
# clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)

In [5]:
# for i in range(20):
#     print(len(clusters[i]))

In [6]:
# #Print for all clusters the top 3 and bottom 3 elements
# for i, cluster in enumerate(clusters):
#     print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
#     for sentence_id in cluster[0:3]:
#         print("\t", corpus_sentences[sentence_id])
#     print("\t", "...")
#     for sentence_id in cluster[-3:]:
#         print("\t", corpus_sentences[sentence_id])

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
def data_loader(data_path):

    f = open(data_path)
    data = json.load(f)

    return data

def column_selection(selected_column, data) :
    empty_column = []
    for i in data:
        empty_column.append(i[selected_column])

    return empty_column

def data_cleaner(txt_data):
    # if limit != None:
    clean_data = []
    empty_indices = []
    for i in range(len(txt_data)):
        if txt_data[i] != '':
                # print(content[i])
            clean_data.append(txt_data[i])
        else :
            empty_indices.append(i)
    # print(empty_indices)
    return clean_data , empty_indices        

def delete_multiple_element(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            # list_object.pop(idx)
            del list_object[idx] 


def index_returner(df,unique_region):
    x = df[df['region'] == unique_region].index
    return x       

def char_counter(x):
    return  len(x)

def token_counter(x):
    return len(x.split())      

In [9]:
import os 
import json
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
# import tensorflow as tf 
# from tensorflow import keras 
# from tensorflow.keras import layers
# from tensorflow.keras.layers import *

plt.style.use('ggplot')

In [10]:
input_path = 'drive/MyDrive/bbc_news_list_uk.json'
data = data_loader(input_path)
print(data[0].keys())

dict_keys(['tags', 'title', 'news_post_date', 'raw_content', 'content', 'url', 'author', 'language', '_id', 'region', 'short_description', 'category', 'crawled_at'])


In [11]:
content = column_selection('content', data)
title = column_selection('title', data)
region  = column_selection('region', data)
short_desc = column_selection('short_description', data)
category = column_selection('category', data)

In [12]:
for i in range(len(region)):
    if region[i] == '':
        region[i] = 'NO_INFO'

In [13]:
clean_content,empty_indices_content = data_cleaner(content)

In [14]:
clean_title,empty_indices_title = data_cleaner(title)
clean_region,empty_indices_region = data_cleaner(region)
clean_short_desc,empty_indices_short_desc = data_cleaner(short_desc)
clean_category,empty_indices_category = data_cleaner(category)

In [15]:
delete_multiple_element(clean_title,empty_indices_content)
delete_multiple_element(clean_region,empty_indices_content)
delete_multiple_element(clean_short_desc,empty_indices_content)
delete_multiple_element(clean_category,empty_indices_content)

In [16]:
df_clean = pd.DataFrame({
    'content': clean_content,
    'title' : clean_title,
    'region' : clean_region,
    'desc' : clean_short_desc,
    'category' : clean_category
})
df_clean.head(1)

,content,title,region,desc,category
0,The heroin substitute methadone can be used as...,Why quitting heroin substitute methadone is 'v...,Health,Heroin substitute methadone is used to wean ad...,BBC News


In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')
corpus_sentences = list(df_clean['content'])
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)


Encode the corpus. This might take a while


Batches:   0%|          | 0/242 [00:00<?, ?it/s]

In [18]:
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.25)

In [19]:
len(clusters)

91